## Install required libraries

<a href="https://colab.research.google.com/github/RedisVentures/redis-google-llms/blob/main/BigQuery_Palm_Redis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install redis google-cloud-aiplatform --upgrade --user

^^^ If prompted press the Restart button to restart the kernel. ^^^

In [ ]:
%%sh
curl -fsSL https://packages.redis.io/gpg | sudo gpg --dearmor -o /usr/share/keyrings/redis-archive-keyring.gpg 
echo "deb [signed-by=/usr/share/keyrings/redis-archive-keyring.gpg] https://packages.redis.io/deb $(lsb_release -cs) main" | sudo tee /etc/apt/sources.list.d/redis.list 
sudo apt-get update  > /dev/null 2>&1
sudo apt-get install redis-stack-server  > /dev/null 2>&1
redis-stack-server --daemonize yes 

## Connect to Redis server

In [ ]:
import os 
import redis
REDIS_HOST = os.getenv("REDIS_HOST", "localhost")
REDIS_PORT = os.getenv("REDIS_PORT", "6379")
REDIS_PASSWORD = os.getenv("REDIS_PASSWORD", "")
#Replace values above with your own if using Redis Cloud instance
#REDIS_HOST="redis-12110.c82.us-east-1-2.ec2.cloud.redislabs.com"
#REDIS_PORT=12110
#REDIS_PASSWORD="pobhBJP7Psicp2gV0iqa2ZOc1WdXXXXX"

#shortcut for redis-cli $REDIS_CONN command
if REDIS_PASSWORD!="":
  os.environ["REDIS_CONN"]=f"-h {REDIS_HOST} -p {REDIS_PORT} -a {REDIS_PASSWORD} --no-auth-warning"
else:
  os.environ["REDIS_CONN"]=f"-h {REDIS_HOST} -p {REDIS_PORT}"
redis = redis.Redis(
  host=REDIS_HOST,
  port=REDIS_PORT,
  password=REDIS_PASSWORD)
redis.ping()

## Authenticate to Google Cloud

In [2]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


In [3]:
project_id = 'central-beach-194106'

## Big Query SQL to Datafame

In [4]:
from google.cloud import bigquery

client = bigquery.Client(project=project_id)

df = client.query('''
SELECT * FROM `bigquery-public-data.hacker_news.full`
where type !='comment' LIMIT 10
''').to_dataframe()

display(df)

,title,url,text,dead,by,score,time,timestamp,type,id,parent,descendants,ranking,deleted
0,None,None,Do you think that requirements were managed we...,<NA>,kvnn,0,1366657379,2013-04-22 19:02:59+00:00,pollopt,5591144,<NA>,<NA>,<NA>,<NA>
1,None,None,Real Analysis by N. L. Carothers,<NA>,niels_olson,0,1297380259,2011-02-10 23:24:19+00:00,pollopt,2204333,<NA>,<NA>,<NA>,<NA>
2,None,None,Here’s Looking at Euclid: A Surprising Excursi...,<NA>,niels_olson,0,1297380259,2011-02-10 23:24:19+00:00,pollopt,2204332,<NA>,<NA>,<NA>,<NA>
3,None,None,Professor Stewart’s Cabinet of Mathematical Cu...,<NA>,niels_olson,0,1297380259,2011-02-10 23:24:19+00:00,pollopt,2204334,<NA>,<NA>,<NA>,<NA>
4,None,None,Advanced Mathematical Methods for Scientists a...,<NA>,niels_olson,0,1297380259,2011-02-10 23:24:19+00:00,pollopt,2204337,<NA>,<NA>,<NA>,<NA>
5,None,None,The Visual Display of Quantitative Information...,<NA>,niels_olson,0,1297380259,2011-02-10 23:24:19+00:00,pollopt,2204336,<NA>,<NA>,<NA>,<NA>
6,Ask HN: Isn't it strange that TFOOT must be pl...,None,"In an HTML &#60;table&#62;, the TFOOT element ...",<NA>,webuiarchitect,0,1298571979,2011-02-24 18:26:19+00:00,story,2259139,<NA>,0,<NA>,<NA>
7,None,None,Other,<NA>,codegeek,0,1428513463,2015-04-08 17:17:43+00:00,pollopt,9342311,<NA>,<NA>,<NA>,<NA>
8,None,None,Elixir,<NA>,justplay,0,1401290371,2014-05-28 15:19:31+00:00,pollopt,7810499,<NA>,<NA>,<NA>,<NA>
9,None,None,eC,<NA>,justplay,0,1401290371,2014-05-28 15:19:31+00:00,pollopt,7810494,<NA>,<NA>,<NA>,<NA>


## Redis Helper functions

In [ ]:
from tqdm.auto import tqdm
tqdm.pandas()

def load_dataframe(redis, df, key_prefix="tweet", id_column="id", pipe_size=100):
    records = df.to_dict(orient="records")
    pipe = redis.pipeline()
    i=1
    for record in tqdm(records):
        i=i+1
        key = f"{key_prefix}:{record[id_column]}"
        pipe.hset(key, mapping=record)
        if (i+1) % pipe_size == 0:
          res=pipe.execute()
    pipe.execute()

## Load Dataframe to Redis

In [ ]:
# clear Redis database (optional)
redis.flushdb()

# load data from Dataframe to Redis HASH
load_dataframe(redis,df,key_prefix="google", pipe_size=100)

## Init Vertex AI

In [5]:
import vertexai

PROJECT_ID = project_id
vertexai.init(project=PROJECT_ID, location="us-central1")

In [8]:
from vertexai.preview.language_models import (ChatModel, InputOutputTextPair,
                                              TextEmbeddingModel,
                                              TextGenerationModel)

In [9]:
generation_model = TextGenerationModel.from_pretrained("text-bison@001")

## Hello Palm!

In [10]:
prompt = "What is a large language model?"

response = generation_model.predict(prompt=prompt)

print(response.text)

A large language model (LLM) is a type of artificial intelligence (AI) model that can understand and generate human language. LLMs are trained on massive datasets of text and code, and they can learn to perform a wide variety of tasks, such as translating languages, writing different kinds of creative content, and answering your questions in an informative way.

LLMs are still under development, but they have the potential to revolutionize many industries. For example, LLMs could be used to create more accurate and personalized customer service experiences, to help doctors diagnose and treat diseases, and to even write entire books and movies.




## Hello Chat!

In [12]:

chat_model = ChatModel.from_pretrained("chat-bison@001")

chat = chat_model.start_chat()

print(
    chat.send_message(
        """
Hello! Can you write a 300 word abstract for a research paper I need to write about the impact of generative AI on society?
"""
    )
)


print(
    chat.send_message(
        """
Could you give me a catchy title for the paper?
"""
    )
)

Generative AI (GAN) is a type of machine learning that uses artificial neural networks to create new, original content. This can include images, text, music, and even videos. GANs have the potential to revolutionize many industries, from healthcare to advertising. However, there are also concerns about the potential negative impacts of GANs, such as the creation of fake news and deepfakes.

In this paper, we explore the potential impact of GANs on society. We first discuss the benefits of GANs, such as their ability to create new and original content, to solve real-world problems, and to democratize creativity
Generative AI: The Future of Creativity
